# Analyses de la plateforme de jeux-video Steam
L'objectif est d'analyser la plateforme afin de trouver la tendance actuelle.
Le jeux de données est disponible à cette adresse : s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json.

In [ ]:
df = spark.read.format("json").load("s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json")

In [ ]:
print(df.count())
display(df.limit(1))

55691


data,id
"List(10, List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP), 13990, Valve, 0, Action, https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513, 999, English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean, Counter-Strike, 5199, 10,000,000 .. 20,000,000, List(true, true, true), 201215, 999, Valve, 2000/11/1, 0, Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role., List(266, 1191, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5426, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 227, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2784, null, null, null, null, null, null, null, null, null, null, null, null, 1607, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 4831, null, null, null, null, null, null, null, null, null, 1707, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 632, null, null, null, null, null, null, null, null, null, null, null, 3392, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 131, null, null, 769, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 881, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 289, null, null, null, 3353, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 614, null, null, null, null, null, null, 304, null, null, null, 1344, null, null, 1864, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1192), game, )",10


In [ ]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

On remarque que le schema est imbriqué donc il faut récupérer les clés pour rendre le schema plus plats.

In [ ]:
from pyspark.sql import functions as F

data_col = [x['name'] for x in df.schema.jsonValue()["fields"][0]["type"]["fields"]]
data_col

Out[38]: ['appid',
 'categories',
 'ccu',
 'developer',
 'discount',
 'genre',
 'header_image',
 'initialprice',
 'languages',
 'name',
 'negative',
 'owners',
 'platforms',
 'positive',
 'price',
 'publisher',
 'release_date',
 'required_age',
 'short_description',
 'tags',
 'type',
 'website']

In [ ]:
from functools import reduce

# On explose le dataframe pour ne plus avoir de schema imbriqué
# et on enlève la colonne id qui est une duplication de app_id
# et data qui a été explosé
games = reduce(
  lambda new_df, col_name: new_df.withColumn(col_name, F.col(f"data.{col_name}")),
  data_col, df
).drop('id').drop('data')

games.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nullable = true)
 |    |-- 1980s: lon

Maintenant que le schema est plus lisible, on peut regarder les données plus facilement

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
games.limit(5).toPandas()

,appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,5199,"10,000,000 .. 20,000,000","{'linux': True, 'mac': True, 'windows': True}",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. ...,"{'1980s': 266.0, '1990's': 1191.0, '2.5D': Non...",game,
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",27,999,PsychoFlux Entertainment,2021/05/14,0,ASCENXION is a 2D shoot 'em up game where you ...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,646,"200,000 .. 500,000","{'linux': False, 'mac': False, 'windows': True}",4032,599,"Team17, NEXT Studios",2020/10/16,0,"Enter a labyrinth that moves as you move, wher...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,
3,1000030,"[Multi-player, Single-player, Co-op, Steam Ach...",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000","{'linux': False, 'mac': True, 'windows': True}",1575,1999,Vertigo Gaming Inc.,2020/10/14,0,"Cook, serve and manage your food truck as you ...","{'1980s': None, '1990's': None, '2.5D': None, ...",game,http://www.cookservedelicious.com
4,1000040,[Single-player],0,DoubleC Games,0,"Action, Casual, Indie, Simulation",https://cdn.akamai.steamstatic.com/steam/apps/...,199,Simplified Chinese,细胞战争,1,"0 .. 20,000","{'linux': False, 'mac': False, 'windows': True}",0,199,DoubleC Games,2019/03/30,0,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,"{'1980s': None, '1990's': None, '2.5D': None, ...",game,


On remarque que la date de sortie et les prix ne sont pas au bon format

In [ ]:
from pyspark.sql.types import IntegerType

games = games.withColumn("release_date", F.to_timestamp(F.col("release_date"), format="y/M/d"))
games = games.withColumn("initialprice", F.col("initialprice") / 100)
games = games.withColumn("price", F.col("price") / 100)
games = games.withColumn("discount", F.col("discount") / 100)
games = games.withColumn("required_age", games.required_age.cast(IntegerType()))

In [ ]:
games.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: double (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: double (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: double (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: timestamp (nullable = true)
 |-- required_age: integer (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nullable = true)
 |    |-- 1980s: